In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#### Steps
- Read Data
- Tokenize using spacy
- 

In [2]:
import sys
import os
LOCAL_FAST_AI = '/Users/krishnakalyan3/Educational/FastAI/fastai/'
AWS_FAST_AI = '/home/ubuntu/fastai'
sys.path.append(AWS_FAST_AI)

In [4]:
from fastai.imports import *
from fastai.transforms import * 
from fastai.conv_learner import * 
from fastai.model import *
from fastai.dataset import * 
from fastai.sgdr import *
from fastai.plots import *
from fastai.learner import *
import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling
from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

In [5]:
PATH = '/home/ubuntu/FastAI_Practice/data/aclImdb/'

In [6]:
!ls {PATH}

imdbEr.txt  imdb.vocab	README	test  train


In [7]:
TRAIN = PATH + 'train/all/'
TEST = PATH + 'test/all/'

In [8]:
TRAIN_FILES = !ls {TRAIN}
TRAIN_FILES[0:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '1000_0.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt']

In [9]:
review = !cat {TRAIN}{TRAIN_FILES[0]}
review[0]

'I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?'

In [10]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [15]:
# BS - Batch Size
# BPTT - This define how many words are processing at a time in each row of the mini-batch

In [11]:
bs=64; 
bptt=70
FILES = dict(train=TRAIN, validation=TEST, test=TEST)

In [12]:
PATH

'/home/ubuntu/FastAI_Practice/data/aclImdb/'

In [13]:
FILES

{'test': '/home/ubuntu/FastAI_Practice/data/aclImdb/test/all/',
 'train': '/home/ubuntu/FastAI_Practice/data/aclImdb/train/all/',
 'validation': '/home/ubuntu/FastAI_Practice/data/aclImdb/test/all/'}

In [14]:
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [18]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(4602, 34945, 1, 20621966)

In [19]:
IMDB_LABEL = data.Field(sequential=False)
splits = torchtext.datasets.IMDB.splits(TEXT, IMDB_LABEL, 'data/')

downloading aclImdb_v1.tar.gz


In [20]:
t = splits[0].examples[0]
t.label, ' '.join(t.text[:16])

('pos',
 'this modern film noir with its off beat humour and dizzying succession of plot twists delivers')

In [28]:
md2 = TextData.from_splits(PATH, splits, bs)
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [30]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=0.3

In [ ]:
m3.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       0.56723  0.53359]                                 
[ 1.       0.35009  0.29712]                                 
[ 2.       0.20719  0.25637]                                 
 22%|██▏       | 84/390 [00:41<02:32,  2.01it/s, loss=0.247]

In [ ]:
accuracy(*m3.predict_with_targs())